In [1]:
import openai
import os
import pandas as pd
import numpy as np
import json

API_KEY = '' # your API Key
openai.api_key = API_KEY

### Load the pre-generate shuffled phrases

In [3]:
Dir_project = '/work/desai-lab/xuanyang/Project/TwoWordTest/'
df_stim = pd.read_csv(os.path.join(Dir_project,'codes','Stim_2160_shuffle10.csv'))
df_stim

,Idx,promptID,idxInPrompt,Stim,shuffleID,shuffleSeed,rand
0,1,1,1,the truck police.,1,3583.0,0.000215
1,2,1,2,the cage rib.,1,3583.0,0.000287
2,3,1,3,the holiday summer.,1,3583.0,0.000488
3,4,1,4,the soap bottle.,1,3583.0,0.001347
4,5,1,5,the radio prison.,1,3583.0,0.001542
...,...,...,...,...,...,...,...
23755,2156,8,159,the goat cheese.,0,NaN,NaN
23756,2157,8,160,the sand beach.,0,NaN,NaN
23757,2158,8,161,the mouth pig.,0,NaN,NaN
23758,2159,8,162,the field flower.,0,NaN,NaN


### Make GPT queries

In [5]:
def ChatGPT_conversation(prompt):
    response = openai.ChatCompletion.create(
        seed = 31415,
        temperature=0,
        model=model_id,
        messages=prompt
        )
    
    api_usage = response['usage']
    print('Total tokens consumed: {0}'.format(api_usage['total_tokens']))
    print(response['choices'][0].finish_reason)
    print(response['choices'][0].finish_reason)
    
    return response

##### Intructions to request discrete ratings

In [9]:
instruct_txt = os.path.join(Dir_project,'codes','instruction_discrete.txt')
with open(instruct_txt) as f:
    instruct = f.readlines()
instruct[0]

"Read each phrase, then judge how meaningful it is as a single concept using a 'makes sense' or 'nonsense' response. If the phrase does not make sense as a single concept, the appropriate answer is 'nonsense'. If the phrase makes sense as a single concept, the appropriate answer is 'makes sense'. For example, 'the goat sky' is 'nonsense'. 'The knife army' is 'nonsense'. 'The soap bubble' is 'makes sense'. 'The computer programmer' is 'makes sense'. Give an answer for each phrase and respond in the format of 'phrase: response'"

##### Intructions to request continuous ratings

In [10]:
instruct_txt = os.path.join(Dir_project,'codes','instruction_continuous.txt')
with open(instruct_txt) as f:
    instruct = f.readlines()
instruct[0]

"Read each phrase, then judge how meaningful it is as a single concept, using a scale from 0 to 4 as follows: If the phrase makes no sense, the appropriate rating is 0. If the phrase makes some sense, the appropriate rating is 2. If the phrase makes complete sense, the appropriate rating is a 4. Please consider the full range of the scale when making your ratings. Examples: 'the goat sky' is 0 (makes no sense), 'the knife army' is 1 (makes very little sense), 'the fox mask' is 2 (makes some sense), 'the soap bubble' is 3 (makes a lot of sense), and 'the computer programmer' is 4 (makes complete sense). Give an answer for each phrase and respond in the format of 'phrase: response'"

#### Loop through models and queries

In [14]:
List_models = ['gpt-3.5-turbo-0125','gpt-3.5-turbo-1106','gpt-4-0125-preview','gpt-4-1106-preview']

In [8]:
for tag in ['continuous','discrete']:
    tag = 'continuous'
    instruct_txt = os.path.join(Dir_project,'codes',f'instruction_{tag}.txt')

    for model_name in List_models:
        Dir_output = os.path.join(Dir_project,'results',tag)
        if not os.path.exists(Dir_output):
            os.mkdir(Dir_output)


        with open(instruct_txt) as f:
            instruct = f.readlines()

        for idx_shuffle in range(0,10):

            df_shuffle = df_stim.loc[df_stim.shuffleID == idx_shuffle].reset_index(drop=True)
            for idx_prompt in df_shuffle.promptID.unique():
                output_json = os.path.join(Dir_output,model_name,f'{model_name}_{tag}_shuffle{idx_shuffle}_prompt{idx_prompt}.json')
                output_txt = os.path.join(Dir_output,model_name,f'{model_name}_{tag}_shuffle{idx_shuffle}_prompt{idx_prompt}.txt')

                if not os.path.exists(output_json):

                    df_prompt = df_shuffle.loc[df_shuffle.promptID == idx_prompt].reset_index(drop=True)
                    lines = '\n'.join(df_prompt.Stim.values)

                    lines = instruct[0]+'\n\n'+lines
                    convo = [{'role': 'system', 'content': lines}]

                    response = ChatGPT_conversation(convo)

                    output = [response['choices'][0]['message']['content']]
                    with open(output_txt, 'a') as f:
                        for line in output:
                            f.write(line)
                    with open(output_json, "w") as outfile:
                        json.dump(response, outfile)
                    print(idx_shuffle,idx_prompt)
        
        

Total tokens consumed: 3501
stop
stop
0 1
Total tokens consumed: 2364
stop
stop
0 2
Total tokens consumed: 3501
stop
stop
0 3
Total tokens consumed: 3497
stop
stop
0 4
Total tokens consumed: 3497
stop
stop
0 5
Total tokens consumed: 3499
stop
stop
0 6
Total tokens consumed: 3501
stop
stop
0 7
Total tokens consumed: 1980
stop
stop
0 8
Total tokens consumed: 3507
stop
stop
1 1
Total tokens consumed: 2358
stop
stop
1 2
Total tokens consumed: 3499
stop
stop
1 3
Total tokens consumed: 2674
stop
stop
1 4
Total tokens consumed: 3503
stop
stop
1 5
Total tokens consumed: 3505
stop
stop
1 6
Total tokens consumed: 2817
stop
stop
1 7
Total tokens consumed: 1982
stop
stop
1 8
Total tokens consumed: 2382
stop
stop
2 1
Total tokens consumed: 2354
stop
stop
2 2
Total tokens consumed: 3491
stop
stop
2 3
Total tokens consumed: 3503
stop
stop
2 4
Total tokens consumed: 3493
stop
stop
2 5
Total tokens consumed: 3491
stop
stop
2 6
Total tokens consumed: 3509
stop
stop
2 7
Total tokens consumed: 1990
stop
s